In [ ]:
# !pip install transformers -U
# !pip install timm -U

In [ ]:
from transformers import DetrImageProcessor, DetrForObjectDetection

image_processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

In [ ]:
model.config.id2label

# Preparing the Images

In [ ]:
from PIL import Image, ImageDraw
import requests
import torch

# url = 'https://nationalhighways.co.uk/media/ndujdy3i/travel-updates-emergency-areas.jpg?anchor=center&mode=crop&width=800&height=533&rnd=132791184246830000'
url = 'https://www.mallofamerica.com//sites/default/files/2017-10/shopping-hero-first_card.jpg'
# if the image is from the Web
if url.startswith('http'):
    image = Image.open(requests.get(url, stream=True).raw)
else:
    # the image is local
    image = Image.open(url)
image

# Detecting Objects

In [ ]:
inputs = image_processor(images = image, 
                         return_tensors = "pt")

In [ ]:
outputs = model(**inputs)

In [ ]:
# create the target size in the format of (height,width)
target_sizes = torch.tensor([image.size[::-1]])

results = image_processor.post_process_object_detection(outputs,
                                                        target_sizes = target_sizes, 
                                                        threshold = 0.9)[0]
results

In [ ]:
draw = ImageDraw.Draw(image)

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
        f"Detected {model.config.id2label[label.item()]} with confidence "
        f"{round(score.item(), 3)} at location {box}"        
    )
    # draw bounding box around object
    draw.rectangle(box, 
                   outline="yellow", 
                   width=2)
    # display the object label
    draw.text((box[0], box[1]-10), 
              model.config.id2label[label.item()], 
              fill="white")  
image

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

objDetector = pipeline("object-detection", 
                       model="facebook/detr-resnet-50")

In [ ]:
if url.startswith('http'):
    image = Image.open(requests.get(url, stream=True).raw)
else:
    # the image is local
    image = Image.open(url)
image

In [ ]:
results = objDetector(image)
results

In [ ]:
# Draw the rectangles on the image
















